### This is a copu of ml-data-preprocessing, written to help with scaffolding for the query multiplexer.
### 02-ml-data-preprocessing is still the main preprocessing file

### ML Data pre-processing
This notebook is for loading and cleaning the data that will be used to train the ML on.
Things like patient heart rate and blood pressure readings that occurred around the time of the administration of the second dose 

It should persist the data into the "out" directory to be consumed by the ml training notebook

In [ ]:
import root_config as rc
from detectdd import config
import pandas as pd

rc.configure()

from detectdd.auth_bigquery import BigQueryClient
from detectdd.serializer import Serializer

print("Loading cohort")

try:
    serializer = Serializer()
    cohort_with_icd = serializer.read_cohort()  # need to run 01-cohort.ipynb to produce the cohort
    print(len(cohort_with_icd))
    cohort_without_icd = serializer.read_cohort_with_no_icd()
    print(len(cohort_without_icd))
    cohort = pd.concat([cohort_with_icd.head(10000), cohort_without_icd.head(10000)])
except FileNotFoundError:
    raise Exception("Need to run [01-cohort.ipynb] at least once to create the cohort file in the /out directory")

big_query = BigQueryClient.auth()


from detectdd.query_multiplexer import WhereClauseGenerator
from detectdd.query_multiplexer import QueryMultiplexer
import pandas as pd
from detectdd.auth_bigquery import BigQueryClient

cohort.describe()

In [ ]:

# Assuming you have a Serializer class that handles reading your saved cohort data
serializer = Serializer()

# Extract unique subject_ids from the cohort data
subject_ids = cohort['subject_id'].unique()

# Convert the list of subject_ids to a format suitable for SQL query
subject_id_str = ', '.join([str(id) for id in subject_ids])
# print(subject_id_str)
# Now, let's proceed to fetch the vital signs for these subject_ids from MIMIC

query_multiplexer = QueryMultiplexer(big_query)

# Write a SQL query to fetch the required vitals where the subject_ids are in your cohort
query = """
SELECT stay_id, subject_id, charttime, heart_rate, sbp, dbp, mbp
FROM `physionet-data.mimiciv_derived.vitalsign`
WHERE ($where) 
    AND (heart_rate IS NOT NULL OR sbp IS NOT NULL OR dbp IS NOT NULL OR mbp IS NOT NULL)
"""

# query = f"""
# SELECT subject_id, heart_rate, sbp, dbp, mbp
# FROM `physionet-data.mimiciv_derived.vitalsign`
# WHERE subject_id IN ({subject_id_str}) limit 100"""

where_fragment = "(stay_id= $stay_id AND charttime > DATETIME_ADD('$dose_b_time', INTERVAL -720 MINUTE) AND charttime < DATETIME_ADD('$dose_b_time', INTERVAL 720 MINUTE))"

multimap_data = {k: v.tolist() for k, v in cohort.groupby('stay_id')['dose_b_time']}
results = query_multiplexer.multiplex_query(query, multi_map_data=multimap_data,
                                            where_clause=WhereClauseGenerator(where_fragment, "stay_id", "dose_b_time"))

In [ ]:
# Run the query
vitals_data = results
vitals_data.describe()

In [ ]:

# Now you have the vital signs data for the patients in your cohort.
# You can proceed to clean this data as needed and use it for further analysis or machine learning model training.

# If you need to save this data locally for further use in your ML training notebook, you can do so like this:
# vitals_data.to_csv(config.out_dir / 'vitals_data.csv', index=False)
vitals_data.to_csv(config.out_dir / 'vitals_data-abc-before-and-after.csv', index=False)

vitals_data.head(100)
